In [1]:
import torch
from torch_geometric.data import Data, Batch
from torch_geometric.utils import to_dense_batch
from copy import deepcopy

In [2]:
in_dim = 21
out_dim = 11
in_dim_edges = 13
out_dim_edges = 17

edge_idx1 = torch.stack([torch.tensor([0, 1, 2, 3, 2]), torch.tensor([1, 2, 3, 0, 0])])
edge_idx2 = torch.stack([torch.tensor([0, 0, 0, 1]), torch.tensor([0, 1, 2, 0])])
x1 = torch.randn(edge_idx1.max() + 1, in_dim, dtype=torch.float32)
e1 = torch.randn(edge_idx1.shape[-1], in_dim_edges, dtype=torch.float32)
x2 = torch.randn(edge_idx2.max() + 1, in_dim, dtype=torch.float32)
e2 = torch.randn(edge_idx2.shape[-1], in_dim_edges, dtype=torch.float32)
# edge_idx1, e1 = add_self_loops(edge_idx1, e1)
# edge_idx2, e2 = add_self_loops(edge_idx2, e2)
g1 = Data(h=x1, edge_index=edge_idx1, edge_attr=e1)
g2 = Data(h=x2, edge_index=edge_idx2, edge_attr=e2)
bg = Batch.from_data_list([deepcopy(g2), g1, g2])

bg

/home/dom/.venv/goli_ipu/lib/python3.8/site-packages/torch_geometric/data/storage.py:261: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_attr', 'h', 'edge_index'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


DataBatch(edge_index=[2, 13], edge_attr=[13, 13], h=[10, 21], batch=[10], ptr=[4])

In [14]:
h = bg.h.clone()
h_dense, mask = to_dense_batch(bg.h, bg.batch, max_num_nodes=13)

print("mask", mask.shape)
print("h_dense", h_dense.shape)
print("h", h.shape)

mask torch.Size([3, 13])
h_dense torch.Size([3, 13, 21])
h torch.Size([10, 21])


In [4]:
h2 = torch.zeros_like(h)
print("h2", h2.shape)
bob = torch.zeros_like(h_dense)
bob = bob.masked_scatter(mask.unsqueeze(-1), h_dense)


h2 torch.Size([10, 21])


In [5]:
h_dense

tensor([[[ 0.5306,  0.0172, -0.5988, -1.2221,  0.1008, -0.6117,  1.1892,
           0.0114,  0.8452, -0.1825,  0.6342, -0.5267, -0.9191,  0.9163,
           0.3338,  0.7765,  0.4692,  0.5633, -0.4402, -0.8481,  1.1564],
         [ 1.3194, -0.0891,  0.9674, -0.3733,  0.9655, -0.6994, -0.0877,
           0.6639, -2.0400,  0.1972, -0.3373,  0.0534, -0.7874,  1.2148,
          -0.0058, -0.8287, -1.4787,  2.0397, -1.3258, -0.7563, -0.3678],
         [ 1.9194,  0.5074,  0.2960,  0.1701,  2.0161,  0.3846,  1.2760,
           0.7141,  0.6438, -0.9007,  0.4849, -0.0110,  0.9367, -2.3675,
           0.2085, -0.0584,  1.7105, -0.1929,  0.6571,  1.9046, -0.9475],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.5802,  0.4303,  1.5639, -1.7533, -0.2877, -0.2405,  0.1214,
          -1.7393,  1.7222,  0.6642, -0.9215,

In [6]:
bob - h_dense

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[-0.5802, -0.4303, -1.5639,  1.7533,  0.2877,  0.2405, -0.1214,
           1.7393, -1.7222, -0.6642,  0.9215,

In [7]:
mask_expand = mask.unsqueeze(-1).expand(h_dense.shape)
mask_idx = (torch.cumsum(mask_expand.flatten(), dim=0) * mask_expand.flatten()).reshape(mask_expand.shape)
mask_idx

tensor([[[  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
           15,  16,  17,  18,  19,  20,  21],
         [ 22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,
           36,  37,  38,  39,  40,  41,  42],
         [ 43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
           57,  58,  59,  60,  61,  62,  63],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0]],

        [[ 64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
           78,  79,  80,  81,  82,  83,  84],
         [ 85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
           99, 100, 101, 102, 103, 104, 105],
         [106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,
          120, 121, 122, 123, 124, 125, 126],
         [127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
          141, 142, 143, 144, 145, 1

In [8]:
print("mask_idx", mask_idx.shape)
print("h_dense", h_dense.shape)


mask_idx torch.Size([3, 4, 21])
h_dense torch.Size([3, 4, 21])


In [9]:
from torch_scatter import scatter

out = scatter(h_dense.flatten(), mask_idx.flatten(), reduce="max")
out2 = scatter(h_dense.flatten(), mask_idx.flatten(), reduce="sum")
print(out.shape)
print(out - out2)
print(out)

torch.Size([211])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([ 0.0000,  0.5306,  0.0172, -0.5988, -1.2221,  0.1008, -0.

In [10]:
h.numel()

210